In [21]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from autogluon.tabular import TabularPredictor

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [23]:
%autoreload
from pipeline import DataPipeline
pipe = DataPipeline()



In [24]:
referance_buildings = pipe.get_reference_buildings_import_data()


In [25]:
referance_buildings
# select these columns from the data
# value	timestamp temperature area wind_speed	wind_direction	cloud_fraction	precipitation

data_for_model = referance_buildings[['value', 'timestamp', 'temperature', 'area', 'wind_speed', 'wind_direction', 'cloud_fraction', 'precipitation']]


In [26]:
import pandas as pd

# Load your dataset
# Assuming you have a DataFrame called 'referance_buildings'
print(data_for_model.size)
data_for_model = data_for_model.dropna()  # Drop missing values if any
print(data_for_model.size)

# Convert timestamp column to datetime and use numerical features
data_for_model['timestamp'] = pd.to_datetime(data_for_model['timestamp'])
# data_for_model['hour'] = data_for_model['timestamp'].dt.hour
# data_for_model['dayofweek'] = data_for_model['timestamp'].dt.dayofweek

# Drop timestamp column as it is not directly usable by the model
data_for_model = data_for_model.drop(columns=['timestamp'])

# Define the target column and features
target = 'value'

# Split into train and test
train_data = data_for_model.sample(frac=0.8, random_state=42)
test_data = data_for_model.drop(train_data.index)

# Train the model using AutoGluon
predictor = TabularPredictor(label=target, eval_metric='mean_absolute_error').fit(
    train_data, 
    # presets='best_quality'
    )


No path specified. Models will be saved in: "AutogluonModels/ag-20241016_094809"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Mon Jul 29 21:14:30 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       12.94 GB / 32.00 GB (40.4%)
Disk Space Avail:   654.40 GB / 926.35 GB (70.6%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. D

Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    13229.24 MB
	Train Data (Original)  Memory Usage: 1.13 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 5 | ['temperature', 'wind_speed', 'wind_direction', 'cloud_fraction', 'precipitation']
		('int', [])   : 1 | ['area']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 5 | ['temperature', 'wind_speed', 'wind_direction', 'cloud_fra

248808
246960


	-7.1041	 = Validation score   (-mean_absolute_error)
	0.01s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBMXT ...


[1000]	valid_set's l1: 7.21816
[2000]	valid_set's l1: 7.18891
[3000]	valid_set's l1: 7.17437
[4000]	valid_set's l1: 7.17716
[5000]	valid_set's l1: 7.18153


	-7.1714	 = Validation score   (-mean_absolute_error)
	41.38s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's l1: 6.74441


	-6.7366	 = Validation score   (-mean_absolute_error)
	9.31s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-6.5242	 = Validation score   (-mean_absolute_error)
	4.83s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost ...
	-6.7183	 = Validation score   (-mean_absolute_error)
	79.19s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-6.4912	 = Validation score   (-mean_absolute_error)
	0.86s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
/opt/homebrew/Caskroom/miniconda/base/envs/ag/lib/python3.10/site-packages/fastai/learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-mod

[1000]	valid_set's l1: 6.69748


	-6.6921	 = Validation score   (-mean_absolute_error)
	12.69s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetTorch': 0.619, 'ExtraTreesMSE': 0.238, 'RandomForestMSE': 0.143}
	-6.3694	 = Validation score   (-mean_absolute_error)
	0.03s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 357.33s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 20126.6 rows/s (2470 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20241016_094809")


In [27]:
# Evaluate on test data
performance = predictor.evaluate(test_data)

print("Evaluation Performance:")
print(performance)  # This will show various metrics such as R^2, RMSE, etc.

# To see feature importance
global_importance = predictor.feature_importance(test_data)
print("\nFeature Importance:")
print(global_importance)  # Shows which features had the most impact on model predictions

Computing feature importance via permutation shuffling for 6 features using 5000 rows with 5 shuffle sets...


Evaluation Performance:
{'mean_absolute_error': -6.451102537680954, 'root_mean_squared_error': -8.755899317452531, 'mean_squared_error': -76.6657728573657, 'r2': 0.4564347759561109, 'pearsonr': 0.6812870538718883, 'median_absolute_error': -4.859660873413086}


	22.18s	= Expected runtime (4.44s per shuffle set)
	8.95s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
                importance    stddev       p_value  n  p99_high   p99_low
area              4.005987  0.089819  3.030537e-08  5  4.190925  3.821049
temperature       2.447230  0.053404  2.719677e-08  5  2.557190  2.337269
wind_direction    0.698313  0.023069  1.427140e-07  5  0.745813  0.650814
wind_speed        0.637683  0.032960  8.534659e-07  5  0.705549  0.569817
cloud_fraction    0.380224  0.026137  2.662699e-06  5  0.434041  0.326407
precipitation     0.236724  0.024285  1.310609e-05  5  0.286726  0.186721
